In [2]:
import pandas as pd 
import numpy as np
from bertopic import BERTopic 
import nltk 
from nltk.corpus import stopwords 
import re
import string

#from nltk.tokenize import word_tokenize
#from nltk.stem import WordNetLemmatizer

In [19]:
#switched to indeed for better interpretation 
df = pd.read_csv("/Users/hanna/Scape-Save-DAEN690/indeed.csv.gz")
df = df[0:300]

C:\Users\hanna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning:

Columns (15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50) have mixed types.Specify dtype option on import or set low_memory=False.



In [20]:
stop_words = set(stopwords.words('english'))
df['description_text'] = df['description_text'].apply(lambda x: str(x).lower())
df['description_text'] = df['description_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
df['description_text'] = df['description_text'].apply(lambda x: ''.join([word for word in x if word.isalpha() or word.isspace()]))
df['description_text'] = df['description_text'].apply(lambda x: x.strip())
#remove punctuation
df['description_text'] = df['description_text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '' , x))
#remove words and digits
df['description_text'] = df['description_text'].apply(lambda x: re.sub('W*dw*','',x))

In [22]:
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(df['description_text'])

Batches: 100%|██████████| 10/10 [00:34<00:00,  3.49s/it]
2023-03-12 12:55:07,005 - BERTopic - Transformed documents to Embeddings
2023-03-12 12:55:09,997 - BERTopic - Reduced dimensionality
2023-03-12 12:55:10,037 - BERTopic - Clustered reduced embeddings


In [23]:
freq = topic_model.get_topic_info(); freq.head(5)
topic_model.get_topic(0)  # Select the most frequent topic

[('security', 0.07041576324841314),
 ('experience', 0.04040577300234382),
 ('information', 0.03979439768303841),
 ('systems', 0.032329151226696044),
 ('management', 0.023382431940169077),
 ('support', 0.023280529833261135),
 ('system', 0.022924571341824258),
 ('cybersecurity', 0.022137335612215557),
 ('work', 0.02197178961115876),
 ('network', 0.021073318355581967)]

In [24]:
topic_model.visualize_barchart(top_n_topics=5)

In [25]:
freq

,Topic,Count,Name
0,-1,121,-1_experience_security_work_team
1,0,101,0_security_experience_information_systems
2,1,48,1_experience_business_work_team
3,2,30,2_security_experience_work_information


In [26]:
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
umap_model = UMAP(n_neighbors=3, n_components=3, min_dist=0.05)
hdbscan_model = HDBSCAN(min_cluster_size=80, min_samples=40,
                        gen_min_span_tree=True,
                        prediction_data=True)

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

stopwords = list(stopwords.words('english'))

# we add this to remove stopwords that can pollute topcs
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stopwords)


In [28]:
model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    top_n_words=5,
    language='english',
    calculate_probabilities=True,
    verbose=True
)
topics, probs = model.fit_transform(df['description_text'])

Batches: 100%|██████████| 10/10 [00:35<00:00,  3.59s/it]
2023-03-12 12:59:42,900 - BERTopic - Transformed documents to Embeddings
2023-03-12 12:59:46,173 - BERTopic - Reduced dimensionality
2023-03-12 12:59:46,196 - BERTopic - Clustered reduced embeddings


In [30]:
topic_model.visualize_barchart()

In [32]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,121,-1_experience_security_work_team
1,0,101,0_security_experience_information_systems
2,1,48,1_experience_business_work_team
3,2,30,2_security_experience_work_information
